### Manipulation of PeaskDB de novo-assisted database search results of Trocas 8 (low water, November 2019) non incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 675) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/680_Trocas8_June2021_PEAKS_138/

/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/680_Trocas8_June2021_PEAKS_138


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 680_Trocas8_June2021_PEAKS_138_peptide.csv     'de novo only peptides.csv'
 680_Trocas8_June2021_PEAKS_138_proteins.csv     peptides_1_1_0.mzid
 680_Trocas8_June2021_PEAKS_138_proteins.fasta   protein-peptides.csv
'DB search psm.csv'


In [5]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup680 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/680_Trocas8_June2021_PEAKS_138/680_Trocas8_June2021_PEAKS_138_peptide.csv")

# remove redundant rows
peaksdb680 = pd.DataFrame.drop_duplicates(peaksdbdup680)

print(peaksdb680.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb680.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb680['#Spec']
del peaksdb680['Accession']
del peaksdb680['PTM']
del peaksdb680['AScore']

mean_length = peaksdb680['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup680))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb680))

#look at the dataframe
peaksdb680.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Sample 6', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Sample 6', 'Accession', 'PTM', 'AScore'],
      dtype='object')
10.428571428571429
# redundant peaksdb peptides in combined dataframe 42
# nonredundant peaksdb peptides in combined dataframe 42


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,LSSPATLNSR,45.56,1044.5564,10,1.6,523.2863,43.48,12700000.0,6,7158,20210623_Trocas8_680_SMCP63_surface_DDA_120min...
1,APVLSDSSC(+57.02)K,44.57,1062.5016,10,-1.3,532.2574,32.74,33800.0,6,4982,20210623_Trocas8_680_SMCP63_surface_DDA_120min...
2,LSSPATLN(+.98)SR,42.07,1045.5404,10,-0.6,523.7772,46.58,160000.0,6,7959,20210623_Trocas8_680_SMCP63_surface_DDA_120min...
3,LATVISPR,37.40,855.5178,8,-0.9,428.7658,57.89,4820000.0,6,9895,20210623_Trocas8_680_SMCP63_surface_DDA_120min...
4,LATVLSPR,37.40,855.5178,8,-0.9,428.7658,57.89,4820000.0,6,9895,20210623_Trocas8_680_SMCP63_surface_DDA_120min...


In [6]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb680['A'] = peaksdb680['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb680['C'] = peaksdb680['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb680['D'] = peaksdb680['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb680['E'] = peaksdb680['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb680['F'] = peaksdb680['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb680['G'] = peaksdb680['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb680['H'] = peaksdb680['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb680 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb680['I'] = peaksdb680['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb680['K'] = peaksdb680['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb680['L'] = peaksdb680['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb680['M'] = peaksdb680['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb680['N'] = peaksdb680['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb680['P'] = peaksdb680['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb680['Q'] = peaksdb680['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb680['R'] = peaksdb680['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb680['S'] = peaksdb680['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb680['T'] = peaksdb680['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb680['V'] = peaksdb680['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb680['W'] = peaksdb680['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb680['Y'] = peaksdb680['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb680['c-carb'] = peaksdb680['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb680['m-oxid'] = peaksdb680['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb680['n-deam'] = peaksdb680['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb680['q-deam'] = peaksdb680['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb680['stripped_peptide'] = peaksdb680['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb680['stripped_length'] = peaksdb680['stripped_peptide'].apply(len)

peaksdb680['NAAF_num.'] = peaksdb680['Area'] / peaksdb680['stripped_length']

# total the number of modifications in sequence
peaksdb680['ptm-total'] = peaksdb680['c-carb'] + peaksdb680['m-oxid'] + peaksdb680['n-deam'] + peaksdb680['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb680['stripped_IL']= peaksdb680['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb680.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/680_SMCP_surf_GF75_PDB.csv")

# check out the results
peaksdb680.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,LSSPATLNSR,45.56,1044.5564,10,1.6,523.2863,43.48,12700000.0,6,7158,...,0,0,0,0,0,LSSPATLNSR,10,1270000.0,0,LSSPATLNSR
1,APVLSDSSC(+57.02)K,44.57,1062.5016,10,-1.3,532.2574,32.74,33800.0,6,4982,...,0,1,0,0,0,APVLSDSSCK,10,3380.0,1,APVLSDSSCK
2,LSSPATLN(+.98)SR,42.07,1045.5404,10,-0.6,523.7772,46.58,160000.0,6,7959,...,0,0,0,1,0,LSSPATLNSR,10,16000.0,1,LSSPATLNSR
3,LATVISPR,37.40,855.5178,8,-0.9,428.7658,57.89,4820000.0,6,9895,...,0,0,0,0,0,LATVISPR,8,602500.0,0,LATVLSPR
4,LATVLSPR,37.40,855.5178,8,-0.9,428.7658,57.89,4820000.0,6,9895,...,0,0,0,0,0,LATVLSPR,8,602500.0,0,LATVLSPR


In [7]:
# keep only stripped peptide I/L and NAAF
db_680 = peaksdb680[['stripped_IL', 'Area', 'NAAF_num.']]

db_680.set_index('stripped_IL')

# write modified dataframe to new txt file
db_680.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/680_SMCP_surf_GF75_PDB_ILnaafs.csv")

db_680.head()

,stripped_IL,Area,NAAF_num.
0,LSSPATLNSR,12700000.0,1270000.0
1,APVLSDSSCK,33800.0,3380.0
2,LSSPATLNSR,160000.0,16000.0
3,LATVLSPR,4820000.0,602500.0
4,LATVLSPR,4820000.0,602500.0


### Exporting txt files of stripped peptides at confidence cutoffs:

In [8]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep680moddup = peaksdb680[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep680dup = peaksdb680[["stripped_peptide"]]

# deduplicate both of these lists
pep680mod = pep680moddup.drop_duplicates()
pep680 = pep680dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 680:", len(pep680moddup))
print("Deduplicated modified PeaksDB peptides in 680:", len(pep680mod))
print("Total stripped PeaksDB peptides in 680:", len(pep680dup))
print("Deduplicated stripped PeaksDB peptides in 680:", len(pep680))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep680.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/680_SMCP_surf_GF75_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/680_SMCP_surf_GF75_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/680_SMCP_surf_GF75_PDB_stripped_peptides.fas

# look at the stripped peptides
pep680.head()

Total modified PeaksDB peptides in 680: 42
Deduplicated modified PeaksDB peptides in 680: 42
Total stripped PeaksDB peptides in 680: 42
Deduplicated stripped PeaksDB peptides in 680: 41


,stripped_peptide
0,LSSPATLNSR
1,APVLSDSSCK
3,LATVISPR
4,LATVLSPR
5,SLVNLGGSK


## NAAF correction and exporting files with AA and PTM totals:

In [9]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb680['A'].sum(),
        'C': peaksdb680['C'].sum(),
        'D': peaksdb680['D'].sum(),
        'E': peaksdb680['E'].sum(),
        'F': peaksdb680['F'].sum(),
        'G': peaksdb680['G'].sum(),
        'H': peaksdb680['H'].sum(),
        'I': peaksdb680['I'].sum(),
        'K': peaksdb680['K'].sum(),
        'L': peaksdb680['L'].sum(),
        'M': peaksdb680['M'].sum(),
        'N': peaksdb680['N'].sum(),
        'P': peaksdb680['P'].sum(),
        'Q': peaksdb680['Q'].sum(),
        'R': peaksdb680['R'].sum(),
        'S': peaksdb680['S'].sum(),
        'T': peaksdb680['T'].sum(),
        'V': peaksdb680['V'].sum(),
        'W': peaksdb680['W'].sum(),
        'Y': peaksdb680['Y'].sum(),
        'c-carb': peaksdb680['c-carb'].sum(),
        'm-oxid': peaksdb680['m-oxid'].sum(),
        'n-deam': peaksdb680['n-deam'].sum(),
        'q-deam': peaksdb680['q-deam'].sum(),
        'Total area': peaksdb680['Area'].sum(),
        'Total length': peaksdb680['stripped_length'].sum()
       }

totalpeaksdb680 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb680['% C w/ carb'] = totalpeaksdb680['c-carb'] / totalpeaksdb680['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb680['% M w/ oxid'] = totalpeaksdb680['m-oxid'] / totalpeaksdb680['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb680['% N w/ deam'] = totalpeaksdb680['n-deam'] / totalpeaksdb680['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb680['% Q w/ deam'] = totalpeaksdb680['q-deam'] / totalpeaksdb680['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb680['NAAF denom.'] = totalpeaksdb680['Total area'] / totalpeaksdb680['Total length']

# write modified dataframe to new txt file
totalpeaksdb680.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/680_SMCP_surf_GF75_PDB_totals.csv")

totalpeaksdb680.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,35,7,15,16,12,30,12,27,27,48,...,2,6,0,53064349.0,411,1.0,0.5,0.26087,0.0,129110.3382


In [10]:
# use the calculated NAAF factor (in totalpeaksdb680 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 129110.3382

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb680['NAAF factor'] = (peaksdb680['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb680_NAAF = peaksdb680[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor SMCP the AAs to normalize its abundance SMCP peak area and peptide length

peaksdb680_NAAF['A-NAAF20'] = peaksdb680_NAAF['A'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['C-NAAF20'] = peaksdb680_NAAF['C'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['D-NAAF20'] = peaksdb680_NAAF['D'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['E-NAAF20'] = peaksdb680_NAAF['E'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['F-NAAF20'] = peaksdb680_NAAF['F'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['G-NAAF20'] = peaksdb680_NAAF['G'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['H-NAAF20'] = peaksdb680_NAAF['H'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['I-NAAF20'] = peaksdb680_NAAF['I'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['K-NAAF20'] = peaksdb680_NAAF['K'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['L-NAAF20'] = peaksdb680_NAAF['L'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['M-NAAF20'] = peaksdb680_NAAF['M'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['N-NAAF20'] = peaksdb680_NAAF['N'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['P-NAAF20'] = peaksdb680_NAAF['P'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['Q-NAAF20'] = peaksdb680_NAAF['Q'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['R-NAAF20'] = peaksdb680_NAAF['R'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['S-NAAF20'] = peaksdb680_NAAF['S'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['T-NAAF20'] = peaksdb680_NAAF['T'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['V-NAAF20'] = peaksdb680_NAAF['V'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['W-NAAF20'] = peaksdb680_NAAF['W'] * peaksdb680['NAAF factor']
peaksdb680_NAAF['Y-NAAF20'] = peaksdb680_NAAF['Y'] * peaksdb680['NAAF factor']

# multiply the NAAF20 factor SMCP the PTMs normalize its abundance SMCP peak area and peptide length

peaksdb680_NAAF['ccarb-NAAF20'] = peaksdb680_NAAF['c-carb'] * peaksdb680_NAAF['NAAF factor']
peaksdb680_NAAF['moxid-NAAF20'] = peaksdb680_NAAF['m-oxid'] * peaksdb680_NAAF['NAAF factor']
peaksdb680_NAAF['ndeam-NAAF20'] = peaksdb680_NAAF['n-deam'] * peaksdb680_NAAF['NAAF factor']
peaksdb680_NAAF['qdeam-NAAF20'] = peaksdb680_NAAF['q-deam'] * peaksdb680_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb680_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/680_SMCP_surf_GF75_PDB_naaf.csv")

peaksdb680_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,LSSPATLNSR,9.836548,1,0,0,0,0,0,0,0,...,9.836548,29.509643,9.836548,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
1,APVLSDSSCK,0.026179,1,1,1,0,0,0,0,0,...,0.000000,0.078537,0.000000,0.026179,0.0,0.0,0.026179,0.0,0.000000,0.0
2,LSSPATLNSR,0.123925,1,0,0,0,0,0,0,0,...,0.123925,0.371775,0.123925,0.000000,0.0,0.0,0.000000,0.0,0.123925,0.0
3,LATVISPR,4.666551,1,0,0,0,0,0,0,1,...,4.666551,4.666551,4.666551,4.666551,0.0,0.0,0.000000,0.0,0.000000,0.0
4,LATVLSPR,4.666551,1,0,0,0,0,0,0,0,...,4.666551,4.666551,4.666551,4.666551,0.0,0.0,0.000000,0.0,0.000000,0.0


In [11]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb680 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb680_NAAF['NAAF factor'].sum(),
        'A': peaksdb680_NAAF['A-NAAF20'].sum(),
        'C': peaksdb680_NAAF['C-NAAF20'].sum(),
        'D': peaksdb680_NAAF['D-NAAF20'].sum(),
        'E': peaksdb680_NAAF['E-NAAF20'].sum(),
        'F': peaksdb680_NAAF['F-NAAF20'].sum(),
        'G': peaksdb680_NAAF['G-NAAF20'].sum(),
        'H': peaksdb680_NAAF['H-NAAF20'].sum(),
        'I': peaksdb680_NAAF['I-NAAF20'].sum(),
        'K': peaksdb680_NAAF['K-NAAF20'].sum(),
        'L': peaksdb680_NAAF['L-NAAF20'].sum(),
        'M': peaksdb680_NAAF['M-NAAF20'].sum(),
        'N': peaksdb680_NAAF['N-NAAF20'].sum(),
        'P': peaksdb680_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb680_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb680_NAAF['R-NAAF20'].sum(),
        'S': peaksdb680_NAAF['S-NAAF20'].sum(),
        'T': peaksdb680_NAAF['T-NAAF20'].sum(),
        'V': peaksdb680_NAAF['V-NAAF20'].sum(),
        'W': peaksdb680_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb680_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb680_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb680_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb680_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb680_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb680_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb680_NAAF['% C w/ carb.'] = totalpeaksdb680_NAAF['c-carb'] / totalpeaksdb680_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb680_NAAF['% M w/ oxid'] = totalpeaksdb680_NAAF['m-oxid'] / totalpeaksdb680_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb680_NAAF['% N w/ deam'] = totalpeaksdb680_NAAF['n-deam'] / totalpeaksdb680_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb680_NAAF['% Q w/ deam'] = totalpeaksdb680_NAAF['q-deam'] / totalpeaksdb680_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb680_NAAF['NAAF check'] = totalpeaksdb680_NAAF['NAAF'] / 129110.3382

# write modified dataframe to new txt file, same name + totals
totalpeaksdb680_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/680_SMCP_surf_GF75_PDB_naaf_totals.csv")

totalpeaksdb680_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,48.948191,30.892143,0.290309,0.709193,0.923693,0.077306,1.032474,0.7392,17.25767,0.191828,...,0.047578,0.290309,0.003336,0.142314,0.0,1.0,0.147865,0.013268,0.0,0.000379
